### Hospital Financial Data Preparation

This notebook cleans and validates hospital level financial and operational data.
The output is a clean dataset used for margin and cost driver analysis.

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


Load raw hospital data from Excel.


In [3]:
file_path = "data/hospital_data.xlsx"
df = pd.read_excel("hafd2024pivot.xlsm", sheet_name="Data")

df.shape


(438, 247)

In [5]:
df.head()

,FAC_NO,FAC_NAME,BEG_DATE,END_DATE,DAY_PER,county name,HSA name,HFPA,TYPE_CNTRL,SYSTEM (≥ 3 HOSP),TYPE_HOSP,TYPE_CARE,TEACH_RURL,DSH hosp,OWNER,lic bed size,ltc range,PHONE,ADDRESS,CITY,ZIP_CODE,CEO,ER_DESIG,BED_LIC,lic bed days,BED_AVL,avail bed days,BED_STF,occ beds,adj occ beds,DAY_MCAR_TR,DAY_MCAR_MC,DAY_MCAL_TR,DAY_MCAL_MC,DAY_CNTY,DAY_THRD_TR,DAY_THRD_MC,DAY_OTH_IND,DAY_OTH,DAY_TOT,adj pat days,mcar t adj day,mcar mc adj day,mcal t adj day,mcal mc adj day,co adj day,third t adj day,third mc adj day,oth ind adj day,oth adj day,DIS_MCAR_TR,DIS_MCAR_MC,DIS_MCAL_TR,DIS_MCAL_MC,DIS_CNTY,DIS_THRD_TR,DIS_THRD_MC,DIS_OTH_IND,DIS_OTH,DIS_TOT,BED_ACUTE,BED_PSYCH,BED_CHEM,BED_REHAB,BED_LTC,BED_RESDNT,acute bed days,psych bed days,chem bed days,rehab bed days,ltc bed days,res bed days,DAY_ACUTE,DAY_PSYCH,DAY_CHEM,DAY_REHAB,DAY_LTC,% ltc days,DAY_RESDNT,DIS_ACUTE,DIS_PSYCH,DIS_CHEM,DIS_REHAB,DIS_LTC,DIS_RESDNT,BAS_NURSRY,DAY_NURSRY,DIS_NURSRY,VIS_MCAR_TR,VIS_MCAR_MC,VIS_MCAL_TR,VIS_MCAL_MC,VIS_CNTY,VIS_THRD_TR,VIS_THRD_MC,VIS_OTH_IND,VIS_OTH,VIS_TOT,VIS_ER,VIS_CLIN,VIS_HOME,VIS_REF_OP,DAY_PIPS,OP_ROOM,OP_MIN_IP,OP_MIN_OP,SURG_IP,SURG_OP,NAT_BIRTHS,C_SECTIONS,GR_PT_REV,DED_FR_REV,TOT_CAP_REV,NET_PT_REV,OTH_OP_REV,total op rev,TOT_OP_EXP,NET_FRM_OP,NONOP_REV,NONOP_EXP,INC_TAX,EXT_ITEM,NET_INCOME,GR_IP_MCAR_TR,GR_IP_MCAR_MC,GR_IP_MCAL_TR,GR_IP_MCAL_MC,GR_IP_CNTY,GR_IP_THRD_TR,GR_IP_THRD_MC,GR_IP_OTH_IND,GR_IP_OTH,GR_IP_TOT,GR_OP_MCAR_TR,GR_OP_MCAR_MC,GR_OP_MCAL_TR,GR_OP_MCAL_MC,GR_OP_CNTY,GR_OP_THRD_TR,GR_OP_THRD_MC,GR_OP_OTH_IND,GR_OP_OTH,GR_OP_TOT,C_ADJ_MCAR_TR,C_ADJ_MCAR_MC,C_ADJ_MCAL_TR,C_ADJ_MCAL_MC,DISP_855,C_ADJ_CNTY,C_ADJ_THRD_TR,C_ADJ_THRD_MC,BAD_DEBT,CHAR_HB,CHAR_OTH,SUB_INDGNT,DED_OTH,CAP_REV_MCAR,CAP_REV_MCAL,CAP_REV_CNTY,CAP_REV_THRD,NETRV_MCAR_TR,NETRV_MCAR_MC,NETRV_MCAL_TR,NETRV_MCAL_MC,NETRV_CNTY,NETRV_THRD_TR,NETRV_THRD_MC,NETRV_OTH_IND,NETRV_OTH,DISP_TRNFR,INTER_TFR,CONTRIBTNS,INC_INVEST,DIST_REV,CNTY_APPRO,EXP_DLY,EXP_AMB,EXP_ANC,EXP_PIP,EXP_POP,EXP_RES,EXP_ED,EXP_GEN,EXP_FISC,EXP_ADM,EXP_UNASSG,EXP_SAL,EXP_BEN,EXP_PHYS,EXP_OTHPRO,EXP_SUPP,EXP_PURCH,EXP_DEPRE,EXP_LEASES,EXP_INSUR,EXP_INTRST,EXP_OTH,CUR_ASST,ASST_LIMTD,NET_PPE,CONST_PROG,INV_OTH,INTAN_ASST,TOT_ASST,CUR_LIAB,DEF_CRED,NET_LTDEBT,EQUITY,LIAB_EQ,CASH,ACCTS_REC,ALLOW_UNCOLL,TOT_PPE,MORT_PAY,BOND_PAY,INTER_REC,INTER_PAY,PROD_HRS,PAID_HRS,HOSP_FTE,STDNT_FTE,PRD_HR_MGT,PRD_HR_TCH,PRD_HR_RN,PRD_HR_LVN,PRD_HR_AID,PRD_HR_CLR,PRD_HR_ENV,PRD_HR_OTH,CNT_HR_RN,CNT_HR_OTH,PRD_HR_DLY,PRD_HR_AMB,PRD_HR_ANC,PRD_HR_ED,PRD_HR_GEN,PRD_HR_FIS,PRD_HR_ADM,exp mcare tr,exp mcare mc,exp mcal tr,exp mcal mc,exp cip,exp 3rd tr,exp 3rd mc,exp other ind,exp other
0,106580996,ADVENTIST HEALTH AND RIDEOUT,2024-01-01,2024-12-31,366,Yuba,02 - GOLDEN EMPIRE,227,Non-Profit,ADVENTIST HEALTH SYSTEMS,Comparable,General Acute,NaN,DSH,ADVENTIST HEALTH,200 - 299,No LTC,530-749-4545,726 4TH STREET,MARYSVILLE,95901,CHRIS CHAMPLIN,3,221,80886,221,80886,151,147.89,269.05,23029,6401,2783,14208,0,2073,4460,91,1081,54126,"98,473.06","37,620.01","10,152.33","4,297.55","29,205.43",0.00,"3,574.58","11,019.82",232.01,"2,460.95",4239,1181,534,3346,0,532,1254,23,263,11372,221,0,0,0,0,0,80886,0,0,0,0,0,54126,0,0,0,0,0.00,0,11372,0,0,0,0,0,14,2688,1438,56128,14863,6837,63239,0,13876,28299,610,5270,189122,73221,14000,0,103860,0,6,230655,258390,1492,2582,1320,560,2257980642,1775129360,0,482851282,12788204,495639486,525727747,-30088261,1293142,19948822,0,0,-48743941,531428648,143954791,57581847,307510385,0,69234499,110640254,1799163,18955929,1241105516,336709489,84365136,31336918,324596322,0,50150251,162730966,2787871,24198173,1016875126,680582369,150827262,73492358,522443290,-1054326,0,89375197,204654947,11680517,0,13604871,0,29522875,0,0,0,0,183705533,76345628,15722692,103294772,0,28327610,61919170,1,13535876,0,-10282003,0,15347,0,0,100566904,48712983,174503039,0,0,0,0,54313300,8038230,88720625,50872666,180714442,59700766,43212586,4087808,70780954,66034712,13035374,6662952,1811008,26732503,52954642,159130127,0

In [10]:
df.info

<bound method DataFrame.info of         FAC_NO                                           FAC_NAME   BEG_DATE  \
0    106580996                       ADVENTIST HEALTH AND RIDEOUT 2024-01-01   
1    106150788                       ADVENTIST HEALTH BAKERSFIELD 2024-01-01   
2    106171049                         ADVENTIST HEALTH CLEARLAKE 2024-01-01   
3    106150706                            ADVENTIST HEALTH DELANO 2024-01-01   
4    106190323                          ADVENTIST HEALTH GLENDALE 2024-01-01   
..         ...                                                ...        ...   
433  106444013                     WATSONVILLE COMMUNITY HOSPITAL 2024-01-01   
434  106301379                        WEST ANAHEIM MEDICAL CENTER 2024-01-01   
435  106190883                   WHITTIER HOSPITAL MEDICAL CENTER 2023-07-01   
436  106571086                         WOODLAND MEMORIAL HOSPITAL 2023-07-01   
437  106380939  ZUCKERBERG SAN FRANCISCO GENERAL HOSPITAL & TR... 2023-07-01   

      E

Keep only columns needed for cost, volume, and margin analysis.


In [11]:
cols_to_keep = [
    "FAC_NO",
    "FAC_NAME",
    "county name",
    "TYPE_HOSP",
    "TEACH_RURL",
    "lic bed size",
    "BED_LIC",
    "BED_STF",
    "occ beds",
    "DAY_TOT",
    "DIS_TOT",
    "TOT_OP_EXP",
    "NET_PT_REV",
    "NET_INCOME"
]

df_clean = df[cols_to_keep].copy()
df_clean.shape

(438, 14)

In [12]:
df_clean.columns = [
    "facility_id",
    "facility_name",
    "county",
    "hospital_type",
    "teaching_rural",
    "bed_size_category",
    "licensed_beds",
    "staffed_beds",
    "occupied_beds",
    "patient_days",
    "discharges",
    "operating_expense",
    "net_patient_revenue",
    "net_income"
]

df_clean.head()


,facility_id,facility_name,county,hospital_type,teaching_rural,bed_size_category,licensed_beds,staffed_beds,occupied_beds,patient_days,discharges,operating_expense,net_patient_revenue,net_income
0,106580996,ADVENTIST HEALTH AND RIDEOUT,Yuba,Comparable,NaN,200 - 299,221,151,147.89,54126,11372,525727747,482851282,-48743941
1,106150788,ADVENTIST HEALTH BAKERSFIELD,Kern,Comparable,NaN,300 - 499,301,198,198.63,72699,17945,704834525,698464738,-5197092
2,106171049,ADVENTIST HEALTH CLEARLAKE,Lake,Comparable,Rural,1 - 49,25,18,15.66,5731,1497,209729803,183332113,-17011138
3,106150706,ADVENTIST HEALTH DELANO,Kern,Comparable,NaN,150 - 199,156,69,64.20,23496,2614,111157919,100703448,220351
4,106190323,ADVENTIST HEALTH GLENDALE,Los Angeles,Comparable,NaN,500 +,515,313,310.87,113777,19036,615072638,563092249,-44826471


Convert numeric columns and handle invalid values.


In [13]:
numeric_cols = [
    "licensed_beds",
    "staffed_beds",
    "occupied_beds",
    "patient_days",
    "discharges",
    "operating_expense",
    "net_patient_revenue",
    "net_income"
]

for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

df_clean.dtypes


facility_id              int64
facility_name           object
county                  object
hospital_type           object
teaching_rural          object
bed_size_category       object
licensed_beds            int64
staffed_beds             int64
occupied_beds          float64
patient_days             int64
discharges               int64
operating_expense        int64
net_patient_revenue      int64
net_income               int64
dtype: object

In [14]:
df_clean.head()

,facility_id,facility_name,county,hospital_type,teaching_rural,bed_size_category,licensed_beds,staffed_beds,occupied_beds,patient_days,discharges,operating_expense,net_patient_revenue,net_income
0,106580996,ADVENTIST HEALTH AND RIDEOUT,Yuba,Comparable,NaN,200 - 299,221,151,147.89,54126,11372,525727747,482851282,-48743941
1,106150788,ADVENTIST HEALTH BAKERSFIELD,Kern,Comparable,NaN,300 - 499,301,198,198.63,72699,17945,704834525,698464738,-5197092
2,106171049,ADVENTIST HEALTH CLEARLAKE,Lake,Comparable,Rural,1 - 49,25,18,15.66,5731,1497,209729803,183332113,-17011138
3,106150706,ADVENTIST HEALTH DELANO,Kern,Comparable,NaN,150 - 199,156,69,64.20,23496,2614,111157919,100703448,220351
4,106190323,ADVENTIST HEALTH GLENDALE,Los Angeles,Comparable,NaN,500 +,515,313,310.87,113777,19036,615072638,563092249,-44826471


In [15]:
df_clean.isna().sum()


facility_id              0
facility_name            0
county                   2
hospital_type            0
teaching_rural         338
bed_size_category        2
licensed_beds            0
staffed_beds             0
occupied_beds            0
patient_days             0
discharges               0
operating_expense        0
net_patient_revenue      0
net_income               0
dtype: int64

Standardize missing categorical values.


In [16]:
df_clean["county"] = df_clean["county"].fillna("Unknown")
df_clean["bed_size_category"] = df_clean["bed_size_category"].fillna("Unknown")
df_clean["teaching_rural"] = df_clean["teaching_rural"].fillna("Non-Teaching")

In [17]:
df_clean.describe()

,facility_id,licensed_beds,staffed_beds,occupied_beds,patient_days,discharges,operating_expense,net_patient_revenue,net_income
count,438.00,438.00,438.00,438.00,438.00,438.00,438.00,438.00,438.00
mean,"106,285,623.14",218.53,148.10,140.27,"50,929.27","7,677.73","479,345,593.45","467,574,434.07","40,629,398.27"
std,"136,355.93",219.09,180.46,168.77,"61,872.31","8,575.97","1,120,944,173.57","1,080,874,419.91","167,580,959.90"
min,"106,010,735.00",0.00,0.00,0.00,0.00,0.00,"4,614,654.00",0.00,"-309,897,047.00"
25%,"106,190,394.00",64.00,41.00,37.22,"13,546.25",932.75,"52,933,003.25","49,229,818.50","-2,226,205.00"
50%,"106,301,290.00",153.00,96.00,87.60,"31,202.50","4,066.50","175,928,872.00","171,871,521.00","3,461,539.50"
75%,"106,374,084.25",308.50,197.75,186.89,"67,536.75","12,076.25","561,685,730.00","548,821,133.75","34,697,527.50"
max,"106,580,996.00","1,500.00","1,450.00","1,339.14","490,125.00","45,014.00","14,465,420,610.00","14,087,743,886.00","2,123,717,585.00"


Check ranges and flag obvious data issues.


In [18]:
df_clean["flag_zero_patient_days"] = df_clean["patient_days"] == 0
df_clean["flag_zero_beds"] = df_clean["licensed_beds"] == 0
df_clean["flag_negative_income"] = df_clean["net_income"] < 0

df_clean[
    ["flag_zero_patient_days", "flag_zero_beds", "flag_negative_income"]
].sum()


flag_zero_patient_days      2
flag_zero_beds              2
flag_negative_income      136
dtype: int64

Save cleaned dataset for downstream analysis.


In [20]:
df_clean.to_csv("hospital_clean_base.csv", index=False)
